In [1]:
# !pip install -U scikit-learn

In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datos_2c2020_tp2.models.log.Logger as logger

In [3]:
train = pd.read_csv("../datasets/xgb-train.csv")
test = pd.read_csv("../datasets/xgb-test.csv")

In [4]:
# train['Week_Day'].describe()

# TP2 - XGBoost Binary Classifier


Luego verifico que dropear filas que se overlapean con test implica
dejar de lado la mayoria de datos por lo cual no se tendraa en cuenta.
(djo el codigo de dropeo comentaado)

-----------------------------------------


# Seteo de features

In [5]:
target = "target"
features = list(train.columns)
features.remove(target)
features.remove("Opportunity_ID")

#VANILLA NO
# features.remove("delivery_delay")
# features.remove("opportunity_lifetime")
# features.remove("converted_taxable_amount")
# features.remove("last_modified_to_delivery")
# features.remove("currency_conversion_rate")
# features.remove("Occur")
# features.remove("delivery_window")
# features.remove("account_creation_to_created_opp")

#VANILLA SI
features.remove('Total_Taxable_Amount')
features.remove('ASP_(converted)')
features.remove("ASP")
features.remove("Total_Amount")
features.remove("Delivery_Year")
# features.remove("Week_Day")

#CONSIDERO REMOVIBLES
# features.remove("created_blocknum")   #FECHA!!
features.remove("late_delivery_blocknum")
features.remove("early_delivery_blocknum")
features.remove("last_modified_blocknum")
features.remove("account_creation_blocknum") #OJO CCON HIPOTESIS DEL CLIENTE VIEEJO
print("Features totales: {}".format(len(features)))



Features totales: 115


# Armado de sets
FORMAS:

In [6]:
X_train, y_train = train.loc[:,features], train.loc[:,target]
X_test_Opp = test.loc[:,"Opportunity_ID"]
X_test = test.loc[:,features]

print("Set de entrenamiento (size) {}".format(X_train.shape))
print("Set de testing (size) {}".format(X_test.shape))

Set de entrenamiento (size) (16947, 115)
Set de testing (size) (2551, 115)


Por otra parte seepaaro el test de entrenamiento en aprox~(80%/20%) segun la fecha de creeacion de la oportunidad.

In [7]:
percent_80 = train["created_blocknum"].describe()['75%']/0.945
beta_test = train[train["created_blocknum"]>percent_80]
beta_train = train[train["created_blocknum"]<percent_80]

In [8]:
A_train, b_train = beta_train.loc[:,features], beta_train.loc[:,target]
# A_test_Opp = beta_test.loc[:,"Opportunity_ID"]
A_test, b_test = beta_test.loc[:,features], beta_test.loc[:,target]

print("Set de entrenamiento secundario (size) {}".format(A_train.shape))
print("Set de testing secundario (size) {}".format(A_test.shape))

Set de entrenamiento secundario (size) (14078, 115)
Set de testing secundario (size) (2869, 115)


In [9]:

dtrain = xgb.DMatrix(X_train, label=y_train)



# Tuneo de hiperparametros
* GRIDSEARCH + CROSS VALIDATION

Configuracion

In [10]:
#------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------
#----------------------------- C O N F I G U R A C I O N ----------------------------------
#------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------

nfolds = 7
rounds = 160
early_stopping_rounds = 10

params = {"objective": "binary:logistic",
          'colsample_bytree': 0.3,
          'subsample': 1.0,
          'learning_rate': 0.1,
          'max_depth': 6,
          'min_child_weight': 1,
          'eval_metric': 'logloss'}
beta_params = params.copy()
beta_params['n_estimators'] = 100


enaable_gridsearch_for_tree = True
enaable_gridsearch_for_sampling = True
enaable_gridsearch_for_learning = True
final_cv = True
enable_parcial_training = True

print("Usando nfolds={},  num_boost_round={}"
      "  y  early_stopping_rounds={}".format(nfolds, rounds, early_stopping_rounds))

#------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------

Usando nfolds=7,  num_boost_round=160  y  early_stopping_rounds=10


In [11]:
#FUNCIONES

def find_best_param_tuple(dtrain, params, param_tuple):
    # Define initial best params and LogLoss
    min_logloss = float("Inf")
    best_params = None
    for param0, param1 in param_tuple['grid']:
        print("CV with {}={}, {}={}".format(param_tuple['names'][0],
                                 param0,
                                 param_tuple['names'][1],
                                 param1))
        # Update our parameters
        params[param_tuple['names'][0]] = param0
        if param_tuple['names'][1]: params[param_tuple['names'][1]] = param1
        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=rounds,
            seed=1234,
            nfold=nfolds,
            metrics='logloss',
            early_stopping_rounds=early_stopping_rounds
        )
        # Update best LogLoss
        mean_logloss = cv_results['test-logloss-mean'].min()
        boost_rounds = cv_results['test-logloss-mean'].argmin()
        print("\tLL {} for {} rounds".format(mean_logloss, boost_rounds+1))
        if mean_logloss < min_logloss:
            min_logloss = mean_logloss
            best_params = (param0, param1)
    print("Best {},{}: {}, {}, LogLoss: {}".format(param_tuple['names'][0],param_tuple['names'][1],best_params[0], best_params[1], min_logloss))
    return best_params

TUNEO DE:
* max_depth
* min_child_weight

In [12]:
if enaable_gridsearch_for_tree:

    tree_params_tuple = {
        'names': ('max_depth', 'min_child_weight'),
        'grid': [
            (max_depth, min_child_weight)
            for max_depth in range(20,26,1)
            for min_child_weight in range(1,2)
        ]
    }

    best_tree_params = find_best_param_tuple(dtrain, params, tree_params_tuple)

    params['max_depth'] = best_tree_params[0]
    params['min_child_weight'] = best_tree_params[1]

CV with max_depth=20, min_child_weight=1
	LL 0.15717857142857142 for 135 rounds
CV with max_depth=21, min_child_weight=1
	LL 0.156558 for 157 rounds
CV with max_depth=22, min_child_weight=1
	LL 0.15760085714285715 for 148 rounds
CV with max_depth=23, min_child_weight=1
	LL 0.15631257142857144 for 132 rounds
CV with max_depth=24, min_child_weight=1
	LL 0.1575902857142857 for 148 rounds
Best max_depth,min_child_weight: 23, 1, LogLoss: 0.15631257142857144


TUNEO DE:
* subsamble
* colsample_bytree


In [13]:
if enaable_gridsearch_for_sampling:

    sample_params_tuple = {
        'names': ('subsample', 'colsample_bytree'),
        'grid': [
            (subsample, colsample_bytree)
            for subsample in [i/10. for i in range(8,11)]
            for colsample_bytree in  [i/10. for i in range(3,5)]
        ]
    }

    best_sample_params = find_best_param_tuple(dtrain, params, sample_params_tuple)

    params['subsample'] = best_sample_params[0]
    params['colsample_bytree'] = best_sample_params[1]

CV with subsample=0.8, colsample_bytree=0.3
	LL 0.16312914285714286 for 149 rounds
CV with subsample=0.8, colsample_bytree=0.4
	LL 0.16095371428571428 for 155 rounds
CV with subsample=0.9, colsample_bytree=0.3
	LL 0.1595277142857143 for 159 rounds
CV with subsample=0.9, colsample_bytree=0.4
	LL 0.1609108571428571 for 127 rounds
CV with subsample=1.0, colsample_bytree=0.3
	LL 0.15631257142857144 for 132 rounds
CV with subsample=1.0, colsample_bytree=0.4
	LL 0.15859299999999998 for 128 rounds
Best subsample,colsample_bytree: 1.0, 0.3, LogLoss: 0.15631257142857144


TUNEO DE:
* learning rate (eta)

In [14]:
if enaable_gridsearch_for_learning:
    learning_rate_grid = [
        (learning_rate, empty)
        for learning_rate in [0.2, 0.15, 0.1, 0.05]
        for empty in range(1,2)
    ]
    learning_rate = {
        'names': ('learning_rate',''),
        'grid': learning_rate_grid,
    }
    params['learning_rate'] = find_best_param_tuple(dtrain, params, learning_rate)[0]

CV with learning_rate=0.2, =1
	LL 0.1620764285714286 for 70 rounds
CV with learning_rate=0.15, =1
	LL 0.15898828571428572 for 109 rounds
CV with learning_rate=0.1, =1
	LL 0.15631257142857144 for 132 rounds
CV with learning_rate=0.05, =1
	LL 0.1615062857142857 for 160 rounds
Best learning_rate,: 0.1, 1, LogLoss: 0.15631257142857144


Final cross validation error

In [15]:
if final_cv:
    cv_results = xgb.cv(
                params,
                dtrain,
                num_boost_round=1000,
                seed=1234,
                nfold=len(features),
                metrics='logloss',
                early_stopping_rounds=early_stopping_rounds
            )
    print("Rounds    LogLoss \n  {}.".format(cv_results["test-logloss-mean"].tail(1), cv_results["test-logloss-mean"].tail(1)))
display(cv_results.tail(1))

Rounds    LogLoss 
  175    0.147247
Name: test-logloss-mean, dtype: float64.


,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
175,0.01203,0.000132,0.147247,0.047316


# Entrenamiento

In [20]:
xgb_clf = xgb.XGBClassifier(objective ='binary:logistic',
            eval_metric='logloss',
            use_label_encoder=False,
            colsample_bytree = params['colsample_bytree'],
            subsample = params['subsample'],
            learning_rate = params['learning_rate'],
            max_depth = params['max_depth'],
            n_estimators = 175,
            reg_alpha=0,
            min_child_weight = params['min_child_weight'])


xgb_clf.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=23, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=175, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1.0, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [17]:
preds = [pred[1] for pred in xgb_clf.predict_proba(X_test)]
# preds

# ENTRENAMIENTO PARCIAL
CALCULO DE ERROR EN BASE A PARTICION DE SET DE ENTRENAMIENTO POR FECHA DE CREACION DE LA OPORTUNIDAD

In [26]:
def partial_tune(params, params_to_tune):
    min_logloss = float("Inf")
    max_accuracy = -float("Inf")
    best_params = None
    for param0, param1 in params_to_tune['range']:
        print("{}={}, {}={}".format(params_to_tune['names'][0],
                                    param0,
                                    params_to_tune['names'][1],
                                    param1))
        # Update our parameters
        params[params_to_tune['names'][0]] = param0
        params[params_to_tune['names'][1]] = param1
        # Run CV
        beta_xgb_clf = xgb.XGBClassifier(objective ='binary:logistic', eval_metric='logloss', use_label_encoder=False,
                    colsample_bytree = params['colsample_bytree'],
                    subsample = params['subsample'],
                    learning_rate = params['learning_rate'],
                    max_depth = int(params['max_depth']),
                    n_estimators = params['n_estimators'],
                    reg_alpha=params['alpha'],
                    min_child_weight = params['min_child_weight'])
        beta_xgb_clf.fit(A_train,b_train)
        beta_preds = [pred[1] for pred in beta_xgb_clf.predict_proba(A_test)]
        print("Log loss: {}".format(log_loss(b_test, beta_preds)))

        predictions = [round(value) for value in beta_preds]
        accuracy = accuracy_score(b_test, predictions)
        print("Accuracy: %.2f%%" % (accuracy * 100.0))

        # Update best LogLoss
        mean_logloss = log_loss(b_test, beta_preds)
        if mean_logloss < min_logloss:
            min_logloss = mean_logloss
            best_params = (param0, param1)
        if max_accuracy < accuracy:
            max_accuracy = accuracy
            best_acu_params = (param0, param1)
    print("Best {}: ({}, {}) with LogLoss: {}".format(params_to_tune['names'], best_params[0], best_params[1], min_logloss))
    print("Best {}: ({}, {}) with Accu: {}".format(params_to_tune['names'], best_acu_params[0], best_acu_params[1], max_accuracy))
    return best_params

#-----------------------------------
#-----------------------------------
#-----------------------------------

if enable_parcial_training:
    print("\n TUNEANDO alpha \n")
    aplha_range = [
        (alpha, n_estimator)
        for alpha in range(0,3)
        for n_estimator in range(1,2)
    ]
    best_alpha = partial_tune(beta_params, {'names':('alpha',''),'range':aplha_range})
    beta_params['alpha'] = best_alpha[0]
    # beta_params['n_estimators'] = best_alpha[1]

    print("\n TUNEANDO learning_rate \n")

    learning_rate_range = [
        (learning_rate, empty)
        for learning_rate in [0.14, 0.13, 0.12, 0.11, 0.10, 0.09, 0.08]
        for empty in range(1,2)
    ]
    best_learning_rate = partial_tune(beta_params, {'names':('learning_rate',''),'range':learning_rate_range})
    beta_params['learning_rate'] = best_learning_rate[0]

    print("\n TUNEANDO n_estimators \n")
    n_estimator_range = [
        (n_estimator, empty)
        for n_estimator in range(130,150,2)
        for empty in range(1,2)
    ]
    best_n_estimator = partial_tune(beta_params, {'names':('n_estimators',''),'range':n_estimator_range})
    beta_params['n_estimators'] = best_n_estimator[0]


    print("\n TUNEANDO TREE_PARAMS \n")
    tree_params_range = [
        (max_depth, min_child_weight)
        for max_depth in range(5,9)
        for min_child_weight in range(1,5)
    ]
    best_tree_params = partial_tune(beta_params, {'names':('max_depth','min_child_weight'),'range':tree_params_range})
    beta_params['max_depth'] = int(best_tree_params[0])
    beta_params['min_child_weight'] = int(best_tree_params[1])


    print("\n TUNEANDO sample_params \n")
    sample_grid_params = [
        (subsample, colsample_bytree)
        for subsample in [i/10. for i in range(6,11)]
        for colsample_bytree in  [i/10. for i in range(1,5)]
    ]
    best_sample_params = partial_tune(beta_params, {'names':('subsample','colsample_bytree'),'range':sample_grid_params})
    beta_params['subsample'] = best_sample_params[0]
    beta_params['colsample_bytree'] = best_sample_params[1]




 TUNEANDO alpha 

alpha=0, =1
Log loss: 0.294366705810412
Accuracy: 87.31%
alpha=1, =1
Log loss: 0.2972205101207274
Accuracy: 87.10%
alpha=2, =1
Log loss: 0.30334033257717596
Accuracy: 86.72%
Best ('alpha', ''): (0, 1) with LogLoss: 0.294366705810412
Best ('alpha', ''): (0, 1) with Accu: 0.8731265249215755

 TUNEANDO learning_rate 

learning_rate=0.14, =1
Log loss: 0.29744629388755367
Accuracy: 88.01%
learning_rate=0.13, =1
Log loss: 0.31371784601821007
Accuracy: 87.00%
learning_rate=0.12, =1
Log loss: 0.3042819112856431
Accuracy: 87.49%
learning_rate=0.11, =1
Log loss: 0.3009535538540431
Accuracy: 86.75%
learning_rate=0.1, =1
Log loss: 0.3066552127427132
Accuracy: 86.48%
learning_rate=0.09, =1
Log loss: 0.294366705810412
Accuracy: 87.31%
learning_rate=0.08, =1
Log loss: 0.30074054923240223
Accuracy: 87.31%
Best ('learning_rate', ''): (0.09, 1) with LogLoss: 0.294366705810412
Best ('learning_rate', ''): (0.14, 1) with Accu: 0.8800975949808295

 TUNEANDO n_estimators 

n_estimators=130

In [27]:
beta_xgb_clf = xgb.XGBClassifier(objective ='binary:logistic',
                                eval_metric='logloss',
                                use_label_encoder=False,
                                colsample_bytree = beta_params['colsample_bytree'],
                                subsample = beta_params['subsample'],
                                learning_rate = beta_params['learning_rate'],
                                max_depth = int(beta_params['max_depth']),
                                n_estimators = beta_params['n_estimators'],
                                reg_alpha=beta_params['alpha'],
                                min_child_weight = beta_params['min_child_weight'])
print(beta_xgb_clf)
beta_xgb_clf.fit(X_train,y_train)
beta_preds = [pred[1] for pred in beta_xgb_clf.predict_proba(X_test)]
beta_pred_df = pd.DataFrame(X_test_Opp)
beta_pred_df["Target"] = beta_preds
beta_pred_df = beta_pred_df.drop_duplicates("Opportunity_ID")
beta_pred_df.to_csv("../submits/xgb-clf-tunned-beta.csv", index=False)
display(beta_pred_df)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.3,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.09, max_delta_step=None, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=136, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=0, reg_lambda=None,
              scale_pos_weight=None, subsample=1.0, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)


,Opportunity_ID,Target
0,10689,0.966305
3,10690,0.693322
8,10691,0.390785
9,10692,0.359057
15,10693,0.974250
...,...,...
2545,12364,0.966379
2547,12365,0.247117
2548,12366,0.172353
2549,12367,0.659700


Guardando las predicciones a CSV

In [21]:
pred_df = pd.DataFrame(X_test_Opp)
pred_df["Target"] = preds
pred_df = pred_df.drop_duplicates("Opportunity_ID")
display(pred_df)
pred_df.to_csv("../submits/xgb-clf-cv_tunned.csv",index=False)

,Opportunity_ID,Target
0,10689,0.966765
3,10690,0.677587
8,10691,0.502843
9,10692,0.242120
15,10693,0.980258
...,...,...
2545,12364,0.981698
2547,12365,0.345006
2548,12366,0.176371
2549,12367,0.734186


In [ ]:
print(params)
plt.rcParams['figure.figsize'] = [15, 25]
xgb.plot_importance(xgb_clf)


In [ ]:
# logger.Logger.log_model("xgb-logloss", params, features, cv_score)